In [56]:
import re
import os
import openai
from bruhcolor import bruhcolored as bc

from dotenv import load_dotenv
load_dotenv()

client = openai.OpenAI()

In [57]:
def complete(system_prompt: str, user_prompt: str) -> str:
    try:
        return client.chat.completions.create(
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.5,
            max_tokens=1500,
            model="gpt-3.5-turbo-1106"
        ).choices[0].message.content
    except Exception as e:
        print(f"ERROR: {e}")
        return None

In [58]:
def generate_outline(story_type: str, target_audience: str):
    system_prompt = """
You are an expert story outliner.
You will be provided a type of story and target audience.
Your task is to take these two items and create a story outline that has 5 main plot points.

You will be given the story type surrounded in four hashtags (####). As an example, #### <story type> ####.    
You will be given the target audience surrounded in four dollar signs ($$$$). As an example, $$$$ <story type> $$$$.

Rules:
- You MUST generate a rich story outline with 3 to 5 main plot points.
- You MUST lead main points with a single dash.
- You MUST provide 5 to 10 sub points for each main point that explains the story line for the main point.
- You MUST lead sub points with two dashes.


Example Input:
Story Type:
####
<story type here>
####

Target Audience:
$$$$
<target audience here>
$$$$


Example Output:
- Point one
-- sub point 1-1
-- sub point 1-2
-- sub point 1-3
. . . 
"""

    user_prompt = """
Story Type:
####
${story_type}
####

Target Audience:
$$$$
${target_audience}
$$$$
"""

    response = complete(
        system_prompt=system_prompt,
        user_prompt=user_prompt.replace("${story_type}", story_type).replace("${target_audience}", target_audience)
    ).split("\n")

    response = [val for val in response if val.strip() != ""]

    chapter_points = {}

    current_chapter = ""

    for val in response:
        if re.match(r"\-\s.*", val):
            current_chapter = val.split("- ")[1]
            chapter_points[current_chapter] = []
        elif re.match(r"\-\-\s.*", val):
            chapter_points[current_chapter].append(val.split("-- ")[1])
        else:
            print("ERROR")

    return chapter_points

In [59]:
def chapter_page_generator(story_type: str, target_audience: str, chapter_title: str, chapter_point: str, previous_knowledge: str, chapter_save_path: str, page_number: int) -> str:
    system_prompt = """
You are a expert partial story generator.
You will be provided a series of details regarding a story.
Your task is to take this information and return a single, multi paragraph page to add to the story.

Information That will be provided to you:
- The story title / purpose surrounded by four hashtags: #### <story title / purpose> ####
- The target audience surrounded by four @ symbols: @@@@ <target audience> @@@@
- The title of the current chapter you are generating surrounded by four dollar signs: $$$$ <chapter title> $$$$
- The quick overview of the page you need to generate surrounded in four ateriks: **** <page to create> ****
- Optionally: previous knowledge of the previous page to keep you on track surrounded in four exclamation points: !!!! <previous knowledge> !!!!

Rules / Task:
- Generate a multi paragraph page to add to the story.
- Use the quick overview to generate this page.
- Use the previous knowledge to tie your page into the story more fluidly.
- Remeber the target audience when generating the page.
"""
    
    user_prompt = """
Story title:
####
${story_title}
####

Target Audience:
@@@@
${target_audience}
@@@@

Chapter Title:
$$$$
${chapter_title}
$$$$

Page to Create Overview:
****
${page_to_create}
****

Previous Knowledge
!!!!
${previous_knowledge}
!!!! 
"""

    if previous_knowledge == "":
        previous_knowledge = "No Previous Knowledge"
    
    filled_user_prompt = user_prompt.replace(
        "${story_title}", story_type).replace(
            "${target_audience}", target_audience).replace(
                "${chapter_title}", chapter_title).replace(
                    "${page_to_create}", chapter_point).replace(
                        "${previous_knowledge}", previous_knowledge)

    new_page = complete(
        system_prompt=system_prompt,
        user_prompt=filled_user_prompt
    )
    
    with open(f"{chapter_save_path}\page_{page_number}.txt", "w") as file:
        file.write(f"Story Type: {story_type}\n" + f"Chapter Title: {chapter_title}\n\n" + new_page + "\n")
    
    return previous_knowledge + f"{chapter_point}\n"
    
    

In [60]:
def story_generator(story_type: str, target_audience: str, story_outline: dict[str, list[str]]) -> None:
    
    cleaned_story_type = re.sub(r"[^A-Za-z0-9 ]", "", story_type).replace(" ", "_")
    
    if not os.path.exists("./stories"):
        print(bc("Creating 'stories' base folder", color=82))
        os.mkdir("./stories")
    
    if not os.path.exists(f"./stories/{cleaned_story_type}"):
        print(bc(f"Creating 'stories/{cleaned_story_type}' story folder", color=72))
        os.mkdir(f"./stories/{cleaned_story_type}")
    
    base_path = f"./stories/{cleaned_story_type}"
    
    chapter_number = 0
    
    for chapter, chapter_points in story_outline.items():
        chapter_number += 1
        previous_chapter_knowledge = ""
        cleaned_chapter_name = f"{chapter_number}_" + re.sub(r"[^A-Za-z0-9 ]", "", chapter).replace(" ", "_")
        chapter_page_save_path = f"./stories/{cleaned_story_type}/{cleaned_chapter_name}"
        os.mkdir(chapter_page_save_path)
        for idx, point in enumerate(chapter_points):
                previous_chapter_knowledge = chapter_page_generator(
                    story_type=story_type,
                    target_audience=target_audience,
                    chapter_title=chapter,
                    chapter_point=point,
                    previous_knowledge=previous_chapter_knowledge,
                    chapter_save_path=chapter_page_save_path,
                    page_number=idx
                )

In [61]:
story_type = "An epic adventure about a Goldfish named Finn."
target_audience = "Children aged 5-10"

story_outline = generate_outline(
    story_type=story_type,
    target_audience=target_audience
)

story_generator(
    story_type=story_type,
    target_audience=target_audience,
    story_outline=story_outline
)

Creating 'stories' base folder
Creating 'stories/An_epic_adventure_about_a_Goldfish_named_Finn' story folder
